# Magic Formula Investing: Implementation and Simulation Using Python
## By Zongning (ZiZi) Zhang, Connor Roberts, Xincheng You, and Samuel Lam

### Table of Contents
* Part A: Data Wrangling
    * Filling in data by cross-referencing the annual data with quarterly data
    * Dropping rows of data with missing crucial values 

## Part A: Cleaning the Data

In [9]:
# importing packages that are used throughout the code
import math
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from matplotlib import rcParams

In [10]:
# importing our raw data and viewing a tiny subset of it
# data was downloaded for the Wharton Research Data Services site, using the Compustat Database
# https://wrds-web.wharton.upenn.edu/wrds/support/Data/_003Sample%20Programs/Compustat/compna_translate_vars.cfm?

# df containly quarterly company fundamentals, such as debt, cash, PPE, etc
# Compustat does not contain quarterly EBIT values, which is crucial to our calculations
df=pd.read_csv("QuarterlyRawDataNoEBIT.csv")

# the Compustat database only offers EBIT on an annual basis, so ebit_df 
# contains annual data for each company's EBIT 
ebit_df=pd.read_csv("AnnualRawInputData.csv")

In [12]:
print "Length of Quarterly Data: ", len(df)
print "Length of Annual Data: ", len(ebit_df)
df.head()

Length of Quarterly Data:  512530
Length of Annual Data:  140522


,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,conm,...,cstkq,dlcq,dlttq,ppentq,pstkq,wcapq,costat,mkvaltq,prccq,sic
0,1004,02/28/1990,1989,3,INDL,C,D,STD,AIR,AAR CORP,...,16.070,32.551,72.606,53.428,0,188.100,A,NaN,31.125,5080
1,1004,05/31/1990,1989,4,INDL,C,D,STD,AIR,AAR CORP,...,16.082,33.821,72.329,63.441,0,184.932,A,NaN,21.250,5080
2,1004,08/31/1990,1990,1,INDL,C,D,STD,AIR,AAR CORP,...,16.086,27.427,71.806,63.545,0,189.351,A,NaN,15.875,5080
3,1004,11/30/1990,1990,2,INDL,C,D,STD,AIR,AAR CORP,...,16.086,33.563,71.769,63.075,0,186.955,A,NaN,11.875,5080
4,1004,02/28/1991,1990,3,INDL,C,D,STD,AIR,AAR CORP,...,16.097,11.436,69.020,63.626,0,184.665,A,NaN,12.875,5080


In [13]:
# trimming data for years past 2014
df=df[df['fyearq']<2015]
ebit_df=ebit_df[ebit_df['fyear']<2015]

# eliminating duplicate data that is double-listed under FS and INDL 
ebit_df=ebit_df[ebit_df['indfmt']!="FS"]

# the length of both dataframes should be shortening
print len(df), len(ebit_df)

487706 124596


In [14]:
# extracting the companies that have missing EBIT values
missing_ebit_tics=ebit_df[ebit_df['ebit'].isnull()].tic.unique()

# removing those companies from our datasets 
df = df[~df.tic.isin(missing_ebit_tics)]
ebit_df = ebit_df[~ebit_df.tic.isin(missing_ebit_tics)]

# the length of both dataframes should be shortening
print len(df), len(ebit_df)

350463 87305


In [15]:
# function to get the appropriate annual EBIT value for each row in the quarterly dataframe
def get_row_ebit(row):
    ebit_row = ebit_df[ebit_df['fyear']==row['fyearq']]
    ebit_row = ebit_row[ebit_row['tic']== row['tic']]
    if ebit_row['ebit'].empty:
        return 0
    else: 
        return float(ebit_row['ebit'])

In [16]:
%%time 
# takes a while to run so don't run this every time
# the edited CSVs have been saved and can be loaded in separately 
df['ebit']=df.apply(get_row_ebit,axis=1)

CPU times: user 17min 15s, sys: 4.22 s, total: 17min 20s
Wall time: 17min 21s


In [17]:
# function to get the appropriate annual cash values for 
# rows with missing cash values in the quarterly dataframe
# 
# quarterly cash values are generally missing before 2007
def get_row_cash(row):
    if math.isnan(row['chq']):
        cash_row = ebit_df[ebit_df['datadate']==row['datadate']]
        cash_row = cash_row[cash_row['tic']==row['tic']]
        if cash_row['ch'].empty:
            return float('NaN')
        else: 
            return float(cash_row['ch'])
    else: 
        return row['chq']    

In [18]:
%%time
# also takes around 10-15 minutes
df['chq']=df.apply(get_row_cash,axis=1)

CPU times: user 21min 28s, sys: 3.17 s, total: 21min 31s
Wall time: 21min 33s


In [19]:
# dropping the rows that still have missing cash values in the quarterly dataset
df=df[~np.isnan(df['chq'])]

# # dropping the rows with missing market values (mkvaltq) in the quarterly dataset
df=df[~np.isnan(df['mkvaltq'])]
print len(df)

# export to CSV so we don't have to rerun the cleaning and get_row_ebit/get_row_cash functions everytime
df.to_csv("CleanedQuarterlyEBITCash.csv")
ebit_df.to_csv("CleanedAnnualEBIT.csv")

111708


# Start here to avoid running data-cleaning code above
### (which takes ~30 minutes)

In [20]:
# re-read in the cleaned dataframes
df=pd.read_csv("CleanedQuarterlyEBITCash.csv")
ebit_df=pd.read_csv("CleanedAnnualEBIT.csv")
print len(df), len(ebit_df)

# drop the index column that gets added mysteriously during exporting
df=df.drop(['Unnamed: 0'],axis=1)
ebit_df=ebit_df.drop(['Unnamed: 0'], axis=1)

111708 87305


In [21]:
# get a list of all the unique tickers in our datasets
tics = pd.DataFrame(data=df['tic'].unique(), columns=['ticker'])
tics.to_csv('AllUniqueTickers.csv')

## 1. Establish minimum market capitalization value of 50M

#### Filter out companies that do not meet this minimum

In [22]:
# we write a function here that will be used later once the year of simulation is determined
# year >= 2006, since mkvaltq are not reported before then 

# returns a list of companies that meet the minimum market cap requirement in a given year
min_market_cap = 50
def set_min_market_cap(df, year):
    df_for_year = df[df['fyearq']==year]
    df_for_year = df_for_year[df_for_year['mkvaltq']>min_market_cap]
    return list(df_for_year['tic'])

## 2. Excluding utility and financial companies

In [23]:
# remove SIC Division H Companies: Finance, Insurance, and Real Estate
# https://www.osha.gov/pls/imis/sic_manual.html
df_below_6000=df[df['sic']<6000]
df_above_7000=df[df['sic']>=7000]

df=pd.concat([df_below_6000,df_above_7000])

# still shortening
print len(df), len(ebit_df)

94031 87305


In [24]:
# remove SIC Division E Companies: Transportation, Communications, Electric, Gas, and Sanitary Services
# https://www.osha.gov/pls/imis/sic_manual.html
df_below_4000=df[df['sic']<4000]
df_above_5000=df[df['sic']>=5000]

df=pd.concat([df_below_4000,df_above_5000])

# still shortening
print len(df), len(ebit_df)

83805 87305


## 3. Calculating Earnings Yield: EBIT/EV

In [25]:
# basic function to calculate ratio 1 across a row
# ratio 1 = EBIT / EV
# EBIT = earnings before interest and taxes, after subtracting depreciation and amortization
# EV = enterprise value = market cap + debt - cash
# EV = MKVALTQ + DTQ - CHQ
# MKVALTQ = market cap = value of preferred stock + value of common stock
# MKVALTQ = PSTKQ + CSTKQ 
# DTQ = debt = long-term debt + net current debt
# DTQ = DLTTQ + DLCQ
# CHQ = cash
# ratio 1 = EBIT / (MKVALTQ + DLCQ + DLTTQ - CHQ)
def ratio_one(row):
    if math.isnan(row['mkvaltq']) or math.isnan(row['dlcq']) or math.isnan(row['dlttq']) or math.isnan(row['chq']) or (row['mkvaltq']+row['dlcq']+row['dlttq']-row['chq'])==0: 
        ratio = float('NaN')
    else: 
        ratio=row['ebit']/(row['mkvaltq']+row['dlcq']+row['dlttq']-row['chq'])
    return ratio

In [26]:
%%time
# add the ratio1 column to the dataframe
df['ratio1']=df.apply(ratio_one,axis=1)

CPU times: user 3.87 s, sys: 20.2 ms, total: 3.89 s
Wall time: 3.89 s


## 4. Calculating Return on Capital: EBIT/(NFA + NWC)

In [27]:
# basic function to calculate ratio 2 across a row
# ratio 2 = EBIT / (NFA + NWC)
# EBIT = earnings before interest and taxes, after subtracting depreciation and amortization
# NFA = net fixed assets = net book value of Property Plant and Equipment (PPENTQ)
# NMW = net working capital = working capital - cash = WCAPQ - CHQ
def ratio_two(row):
    if math.isnan(row['ppentq']) or math.isnan(row['wcapq']) or math.isnan(row['chq']) or row['ppentq']+row['wcapq']-row['chq']==0:
        ratio = float('NaN')
    else: 
        ratio=row['ebit']/(row['ppentq']+row['wcapq']-row['chq'])
    return ratio

In [28]:
%%time
# add the ratio2 column to the dataframe
df['ratio2']=df.apply(ratio_two,axis=1)

CPU times: user 3.05 s, sys: 22.2 ms, total: 3.07 s
Wall time: 3.06 s


In [29]:
# our dataframe now has two additional columns: ratio1 and ratio2
df.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,conm,...,ppentq,pstkq,wcapq,costat,mkvaltq,prccq,sic,ebit,ratio1,ratio2
60,1050,12/31/2006,2006,4,INDL,C,D,STD,CECE,CECO ENVIRONMENTAL CORP,...,8.530,0,14.311,A,103.0205,8.97,3564,6.047,0.051216,0.270004
61,1050,03/31/2007,2007,1,INDL,C,D,STD,CECE,CECO ENVIRONMENTAL CORP,...,8.915,0,17.872,A,143.5975,12.49,3564,12.634,0.075565,0.479196
62,1050,06/30/2007,2007,2,INDL,C,D,STD,CECE,CECO ENVIRONMENTAL CORP,...,8.766,0,16.457,A,167.8835,11.48,3564,12.634,0.074792,0.512806
63,1050,09/30/2007,2007,3,INDL,C,D,STD,CECE,CECO ENVIRONMENTAL CORP,...,9.076,0,17.810,A,225.9759,15.28,3564,12.634,0.055902,0.483062
64,1050,12/31/2007,2007,4,INDL,C,D,STD,CECE,CECO ENVIRONMENTAL CORP,...,9.284,0,21.220,A,162.3832,10.98,3564,12.634,0.075910,0.423278


## 5. Ranking the companies based on calculated ratios
### and extracting the top 20-30 companies

In [30]:
# get a sorted datafframe of the eligible companies for a given year

# we can also specify the weights of the two ratios used to create the overall ranking
def get_top(df, year, ratio1weight):
    # eliminate companies that do not meet the min market cap
    short_df = df[df['tic'].isin(set_min_market_cap(df, year))]
    short_df = short_df[short_df['fyearq']==year]
    
    # rank the rows based on ratio1 and ratio2, with a larger ratio ==> better ranking
    short_df['rank1'] = short_df['ratio1'].rank(ascending=False,na_option='bottom')
    short_df['rank2'] = short_df['ratio2'].rank(ascending=False,na_option='bottom')
    
    # sum the two ranks based on the specified ratio1weight parameter
    short_df['ranksum'] = ratio1weight * short_df['rank1']+ (1-ratio1weight) * short_df['rank2']
    
    # get the final ranking from the combined sum
    short_df['finalrank'] = short_df['ranksum'].rank(ascending=True,na_option='bottom')
    sorted_short = short_df.sort(columns='finalrank',ascending=True)
    
    # return a tuple of the sorted dataframe and the sorted list of unique tickers 
    return sorted_short, list(sorted_short.tic.unique())

## 6. Invest in 30 highest-ranked companies
We are simulating a 1,000,000 dollar portfolio. 

In [35]:
# monthly stock price data is downloaded from CRSP database
prices_df=pd.read_csv("CRSPMonthlyPrices.csv")
prices_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,PERMNO,date,SICCD,TICKER,COMNAM,PRC,CFACPR,PRCADJUST,year
0,0,0,0,10001,1/31/90,4920,GFGC,GREAT FALLS GAS CO,-9.9375,3,29.8125,1990
1,1,1,1,10001,2/28/90,4920,GFGC,GREAT FALLS GAS CO,-9.8750,3,29.6250,1990
2,2,2,2,10001,3/30/90,4920,GFGC,GREAT FALLS GAS CO,-9.8750,3,29.6250,1990
3,3,3,3,10001,4/30/90,4920,GFGC,GREAT FALLS GAS CO,-9.8750,3,29.6250,1990
4,4,4,4,10001,5/31/90,4920,GFGC,GREAT FALLS GAS CO,9.7500,3,29.2500,1990


In [36]:
prices_df=prices_df.drop(['Unnamed: 0'],axis=1)
prices_df=prices_df.drop(['Unnamed: 0.1'],axis=1)

In [37]:
prices_df.head()

,PERMNO,date,SICCD,TICKER,COMNAM,PRC,CFACPR,PRCADJUST,year
0,10001,1/31/90,4920,GFGC,GREAT FALLS GAS CO,-9.9375,3,29.8125,1990
1,10001,2/28/90,4920,GFGC,GREAT FALLS GAS CO,-9.8750,3,29.6250,1990
2,10001,3/30/90,4920,GFGC,GREAT FALLS GAS CO,-9.8750,3,29.6250,1990
3,10001,4/30/90,4920,GFGC,GREAT FALLS GAS CO,-9.8750,3,29.6250,1990
4,10001,5/31/90,4920,GFGC,GREAT FALLS GAS CO,9.7500,3,29.2500,1990


In [39]:
# calculate the adjusted price for each row
# adjusted price = absolute price * monthly cumulative adjustment factor 
# PRCADJUST = PRC * CFACPR
def get_adjusted_price(row):
    # some prices are negative to indicate that 
    # they are an average of bid and ask price 
    # because closing price was not available
    if row['PRC']<0:
        return -row['PRC']*row['CFACPR']
    else: 
        return row['PRC']*row['CFACPR']

In [40]:
# add the PRCADJUST column to the dataframe
prices_df['PRCADJUST']=prices_df.apply(get_adjusted_price,axis=1)

In [41]:
import dateutil.parser as parser

# parse out the year from the date value
def get_year(row):
    return parser.parse(row['date']).year

# add the year column to the dataframe
prices_df['year']=prices_df.apply(get_year,axis=1)

In [42]:
# drop rows with missing PRCADJUST values
prices_df=prices_df[~np.isnan(prices_df['PRCADJUST'])]

# save the dataframe to a CSV
prices_df.to_csv("CRSPMonthlyPrices.csv")

In [224]:
def simulate_year(year,ratio1weight,timespan):
    stock_list=get_top(df,year,ratio1weight)[1]
    TOTALNUMSTOCKS=30
    portfolio_size=1000000
    
    amount_per_stock=portfolio_size/TOTALNUMSTOCKS
    # print stock_list
    # for each stock, we want the quantity of stock purchased at end of Jan * (value at end of year - value at beginning of year) 
    num_stocks=0
    total_change=0
    for stock in stock_list:
        stock_prices= prices_df[prices_df['TICKER']==stock]
        stock_prices=stock_prices[stock_prices['year']>=year]
        stock_prices=stock_prices[stock_prices['year']<year+timespan]
        # check that we have data from this specific stock and year
        if len(stock_prices)!=0:
            total_change = total_change+amount_per_stock/stock_prices.head(1).PRCADJUST.values[0] * (stock_prices.tail(1).PRCADJUST.values[0]-stock_prices.head(1).PRCADJUST.values[0])
            num_stocks=num_stocks+1
            if num_stocks>TOTALNUMSTOCKS:
                break
    portfolio_return_size=portfolio_size + total_change
    return portfolio_return_size
    
simulate_year(1998,0.5,10)
    

1196367.3462335477

In [230]:
%%time
for year in range(1995,2015):
    for ratio in [0.375,0.5,0.625,0.75,0.875]:
        print year,ratio, simulate_year(year, ratio,1)

 1995 0.125 1147916.13521
1995 0.25 1152677.99235
1995 0.375 1128667.2897
1995 0.5 1128667.2897
1995 0.625 1141167.1647
1995 0.75 1190519.55305
1995 0.875 1183202.55305
1996 0.125 1081642.31972
1996 0.25 1046449.6496
1996 0.375 1021020.25296
1996 0.5 1041541.61381
1996 0.625 1012462.27973
1996 0.75 1010330.90011
1996 0.875 1007088.14663
1997 0.125 nan
1997 0.25 nan
1997 0.375 1070491.29975
1997 0.5 1069540.20227
1997 0.625 1056459.69878
1997 0.75 1085143.79095
1997 0.875 1077761.94791
1998 0.125 973842.63158
1998 0.25 1053991.36127
1998 0.375 991117.387426
1998 0.5 1015326.25561
1998 0.625 994574.946783
1998 0.75 981201.880038
1998 0.875 1014418.18449
1999 0.125 1080026.43471
1999 0.25 1021265.53632
1999 0.375 955904.415308
1999 0.5 948428.533019
1999 0.625 1116466.94616
1999 0.75 1105587.47862
1999 0.875 1149014.31884
2000 0.125 956216.8359
2000 0.25 1072748.5467
2000 0.375 1063454.8827
2000 0.5 1005766.398
2000 0.625 934543.051216
2000 0.75 1150609.25767
2000 0.875 1161824.37681
2001

In [ ]:
def simulate_year_recursive(portfolio_size, year,ratio1weight,timespan):
    stock_list=get_top(df,year,ratio1weight)[1]
    TOTALNUMSTOCKS=30
    # portfolio_size=1000000
    
    amount_per_stock=portfolio_size/TOTALNUMSTOCKS
    # print stock_list
    # for each stock, we want the quantity of stock purchased at end of Jan * (value at end of year - value at beginning of year) 
    num_stocks=0
    total_change=0
    for stock in stock_list:
        stock_prices= prices_df[prices_df['TICKER']==stock]
        stock_prices=stock_prices[stock_prices['year']>=year]
        stock_prices=stock_prices[stock_prices['year']<year+timespan]
        # check that we have data from this specific stock and year
        if len(stock_prices)!=0:
            total_change = total_change+amount_per_stock/stock_prices.head(1).PRCADJUST.values[0] * (stock_prices.tail(1).PRCADJUST.values[0]-stock_prices.head(1).PRCADJUST.values[0])
            num_stocks=num_stocks+1
            if num_stocks>TOTALNUMSTOCKS:
                break
    portfolio_return_size=portfolio_size + total_change
    return portfolio_return_size
    
simulate_year(1998,0.5,10)